In [94]:
from math import remainder
import pandas as pd
import numpy as np
from sklearn.compose import make_column_transformer

In [95]:
car = pd.read_csv('car.csv')

In [96]:
car.head()

,name,company,year,Price,kms_driven,fuel_type
0,Hyundai Santro Xing XO eRLX Euro III,Hyundai,2007,"80,000","45,000 kms",Petrol
1,Mahindra Jeep CL550 MDI,Mahindra,2006,"4,25,000",40 kms,Diesel
2,Maruti Suzuki Alto 800 Vxi,Maruti,2018,Ask For Price,"22,000 kms",Petrol
3,Hyundai Grand i10 Magna 1.2 Kappa VTVT,Hyundai,2014,"3,25,000","28,000 kms",Petrol
4,Ford EcoSport Titanium 1.5L TDCi,Ford,2014,"5,75,000","36,000 kms",Diesel


In [97]:
car.shape

(892, 6)

In [98]:
car.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 892 entries, 0 to 891
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   name        892 non-null    object
 1   company     892 non-null    object
 2   year        892 non-null    object
 3   Price       892 non-null    object
 4   kms_driven  840 non-null    object
 5   fuel_type   837 non-null    object
dtypes: object(6)
memory usage: 41.9+ KB


In [99]:
car.describe()

,name,company,year,Price,kms_driven,fuel_type
count,892,892,892,892,840,837
unique,525,48,61,274,258,3
top,Honda City,Maruti,2015,Ask For Price,"45,000 kms",Petrol
freq,13,235,117,35,30,440


In [100]:
# year has many non integer values
# price has Ask for price
# km driven has kms with integer
# km driven has some nan values
# fuel type has many nan values
# keep 1st three words of name

## Data Cleaning

In [101]:
backup_car = car.copy()

In [102]:
car = car[car['year'].str.isnumeric()]

In [103]:
car['year'] = car['year'].astype(int)

In [104]:
car.info()

<class 'pandas.core.frame.DataFrame'>
Index: 842 entries, 0 to 891
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   name        842 non-null    object
 1   company     842 non-null    object
 2   year        842 non-null    int64 
 3   Price       842 non-null    object
 4   kms_driven  840 non-null    object
 5   fuel_type   837 non-null    object
dtypes: int64(1), object(5)
memory usage: 46.0+ KB


In [105]:
car = car[car['Price'] != "Ask For Price"]

In [106]:
car['Price'] = car['Price'].str.replace(',','').astype(int)

In [107]:
car['kms_driven'] = car['kms_driven'].str.split(' ').str.get(0)

In [108]:
car['kms_driven'] = car['kms_driven'].str.replace(',','')

In [109]:
car = car[car['kms_driven'].str.isnumeric()]

In [110]:
car['kms_driven'] = car['kms_driven'].astype(int)

In [111]:
car = car[~car['fuel_type'].isna()]

In [112]:
car['name'] = car['name'].str.split(' ').str.slice(0,3).str.join(' ')

In [113]:
car = car.reset_index(drop = True)

In [114]:
car = car[car['Price']<6e6].reset_index(drop = True)

In [115]:
car

,name,company,year,Price,kms_driven,fuel_type
0,Hyundai Santro Xing,Hyundai,2007,80000,45000,Petrol
1,Mahindra Jeep CL550,Mahindra,2006,425000,40,Diesel
2,Hyundai Grand i10,Hyundai,2014,325000,28000,Petrol
3,Ford EcoSport Titanium,Ford,2014,575000,36000,Diesel
4,Ford Figo,Ford,2012,175000,41000,Diesel
...,...,...,...,...,...,...
810,Maruti Suzuki Ritz,Maruti,2011,270000,50000,Petrol
811,Tata Indica V2,Tata,2009,110000,30000,Diesel
812,Toyota Corolla Altis,Toyota,2009,300000,132000,Petrol
813,Tata Zest XM,Tata,2018,260000,27000,Diesel


In [116]:
car.to_csv('Cleaned_Car.csv')

## Model Building

In [117]:
x = car.drop(columns = 'Price')
y = car['Price']

In [118]:
from sklearn.model_selection import train_test_split

In [119]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.2)

In [120]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import make_column_transformer
from sklearn.pipeline import make_pipeline

In [121]:
lr = LinearRegression()
ohe = OneHotEncoder()
ohe.fit(x[['name','company','fuel_type']])

,categories,'auto'
,drop,None
,sparse_output,True
,dtype,<class 'numpy.float64'>
,handle_unknown,'error'
,min_frequency,None
,max_categories,None
,feature_name_combiner,'concat'


In [123]:
column_trans = make_column_transformer((OneHotEncoder(categories=ohe.categories_),['name','company','fuel_type']),remainder = 'passthrough')

In [124]:
pipeline = make_pipeline(column_trans, lr)

In [125]:
pipeline.fit(x_train, y_train)

,steps,"[('columntransformer', ...), ('linearregression', ...)]"
,transform_input,None
,memory,None
,verbose,False
,transformers,"[('onehotencoder', ...)]"
,remainder,'passthrough'
,sparse_threshold,0.3
,n_jobs,None
,transformer_weights,None
,verbose,False
,verbose_feature_names_out,True


In [126]:
y_pred = pipeline.predict(x_test)

In [128]:
r2_score(y_test, y_pred)

0.649194691023089

In [129]:
scores = []
for i in range(1000):
    x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.1, random_state = i)
    lr = LinearRegression()
    pipeline = make_pipeline(column_trans, lr)
    pipeline.fit(x_train, y_train)
    y_pred = pipeline.predict(x_test)
    scores.append(r2_score(y_test, y_pred))

In [130]:
scores[np.argmax(scores)]

0.8991157554877304

In [131]:
import pickle
pickle.dump(pipeline, open('pipeline.pkl', 'wb'))

In [132]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.1, random_state = np.argmax(scores))
lr = LinearRegression()
pipeline = make_pipeline(column_trans, lr)
pipeline.fit(x_train, y_train)
y_pred = pipeline.predict(x_test)
scores.append(r2_score(y_test, y_pred))